In [2]:
import pandas as pd
import numpy as np
from joblib import dump
from sklearn.neighbors import NearestNeighbors
from joblib import dump

In [2]:
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="9e5f390cdd7e44658f17b1b3228c02b2",
                                                           client_secret="a2d7e42715d54e12a3a4e86ba5d93dc0"))

In [ ]:
"""
Primary:
- id (Id of track generated by Spotify)
Numerical:
- acousticness (Ranges from 0 to 1)
- danceability (Ranges from 0 to 1)
- energy (Ranges from 0 to 1)
- duration_ms (Integer typically ranging from 200k to 300k)
- instrumentalness (Ranges from 0 to 1)
- valence (Ranges from 0 to 1)
- popularity (Ranges from 0 to 100)
- tempo (Float typically ranging from 50 to 150)
- liveness (Ranges from 0 to 1)
- loudness (Float typically ranging from -60 to 0)
- speechiness (Ranges from 0 to 1)
- year (Ranges from 1921 to 2020)
Dummy:
- mode (0 = Minor, 1 = Major)
- explicit (0 = No explicit content, 1 = Explicit content)
Categorical:
- key (All keys on octave encoded as values ranging from 0 to 11, starting on C as 0, C# as 1 and so on…)
- artists (List of artists mentioned)
- release_date (Date of release mostly in yyyy-mm-dd format, however precision of date may vary)
- name (Name of the song)"""

In [26]:
urn = 'spotify:track:0cS0A1fUEUd1EW3FcF8AEI'

results = sp.track(urn)
random = pd.json_normalize(results)
random

,artists,available_markets,disc_number,duration_ms,explicit,href,id,is_local,name,popularity,...,album.id,album.images,album.name,album.release_date,album.release_date_precision,album.total_tracks,album.type,album.uri,external_ids.isrc,external_urls.spotify
0,[{'external_urls': {'spotify': 'https://open.s...,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...",1,168333,False,https://api.spotify.com/v1/tracks/0cS0A1fUEUd1...,0cS0A1fUEUd1EW3FcF8AEI,False,Keep A Song In Your Soul,11,...,6os2Mv58OYnQClPf7B9E1s,"[{'height': 640, 'url': 'https://i.scdn.co/ima...",Crazy Blues: The Best Of Mamie Smith,1920,year,25,album,spotify:album:6os2Mv58OYnQClPf7B9E1s,USSM10309151,https://open.spotify.com/track/0cS0A1fUEUd1EW3...


In [5]:
df = pd.read_csv('suggestor/edited_data.csv')

def to_list(df):
    df['combined'] = df['name'] + ' - ' +  df['artists']
    df = df.drop_duplicates(subset='combined', keep='first')
    combined = df['combined']
    track_artist = combined.tolist()
    return track_artist

In [6]:
track_list = to_list(df)

In [7]:
track_list[5]

'Crazy Blues - 78rpm Version - MAMIE SMITH & HER JAZZ HOUNDS'

In [7]:
df = pd.read_csv('suggestor/edited_data.csv')
url = 'http://open.spotify.com/track/' + df['id']
df['url'] = url
    
df['combined'] = df['name'] + ' - ' +  df['artists']
df = df.drop_duplicates(subset='combined', keep='first')

# reordering columns, leaving out ID and release date
df = df[['combined', 'url', 'year', 'acousticness', 'danceability', 'duration_ms', 'energy',
       'explicit', 'instrumentalness', 'key', 'liveness', 'loudness',
       'mode', 'popularity', 'speechiness', 'tempo', 'valence']]

# target set will be both artist and name
y_set = ['combined', 'url']

# droping target from data matrix
df_data = df.drop(y_set, axis=1)

# set target
df_target = df[y_set]

# fit on data, 12 neighbors
nn = NearestNeighbors(algorithm='brute', leaf_size =12, n_neighbors=15, n_jobs=-1)
nn.fit(df_data)

# sample a song(index) from df_data to use as our query point 
input_index=input

# vectorize 
data_vect = [df_data.iloc[input_index].values]
neigh_dist, neigh_indices = nn.kneighbors(data_vect)
indexs = neigh_indices.flat[0:12].tolist()

#adding url to each track
output = df_target.iloc[indexs]

TypeError: Cannot index by location index with a non-integer key

In [30]:
input = 'Shake It Off'
index = df.loc[df.isin([input]).any(axis=1)].index.tolist()
index = index[0]
model = song_model(index) 
model

,artists,name,url
92700,TAYLOR SWIFT,Shake It Off,http://open.spotify.com/track/3fthfkkvy9av3q3u...
74252,JESSE MCCARTNEY,Body Language,http://open.spotify.com/track/7lT9KJpfzMTNi970...
92762,J. COLE,Hello,http://open.spotify.com/track/30Chv2SmIry70Ywt...
172017,"DANIEL MERRIWEATHER', 'ADELE",Water And A Flame (feat. Adele),http://open.spotify.com/track/6FZjUIfR6vEEg1bT...
38094,JASON ALDEAN,Burnin' It Down,http://open.spotify.com/track/5HJqpDspKDKwQpLj...
12912,TALKING HEADS,And She Was,http://open.spotify.com/track/5JwK7v6XhG0YgqDu...
11592,CARPENTERS,Christmas Song (Chestnuts Roasting On An Open ...,http://open.spotify.com/track/177bLiZAvMp5FSLQ...
69471,CELTIC FROST,Into the Crypts of Rays,http://open.spotify.com/track/4xQO8MVtFZOW2KJM...
136837,PATRICK BRUEL,J'te l'dis quand même,http://open.spotify.com/track/1Ek1OEhcEw9gRr5f...
72350,BECK,Sexx Laws,http://open.spotify.com/track/1ELY3KTgI83N1k0x...


In [8]:
# reading csv
df = pd.read_csv('suggestor/edited_data.csv')

In [9]:
df

,Unnamed: 0,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0,0.991000,MAMIE SMITH,0.598,168333,0.224,0,0cS0A1fUEUd1EW3FcF8AEI,0.000522,5,0.3790,-12.628,0,Keep A Song In Your Soul,12,1920,0.0936,149.976,0.6340,1920
1,1,0.643000,SCREAMIN' JAY HAWKINS,0.852,150200,0.517,0,0hbkKFIJm7Z05H8Zl9w30f,0.026400,5,0.0809,-7.261,0,I Put A Spell On You,7,1920-01-05,0.0534,86.889,0.9500,1920
2,2,0.993000,MAMIE SMITH,0.647,163827,0.186,0,11m7laMUgmOKqI3oYzuhne,0.000018,0,0.5190,-12.098,1,Golfing Papa,4,1920,0.1740,97.600,0.6890,1920
3,3,0.000173,OSCAR VELAZQUEZ,0.730,422087,0.798,0,19Lc5SfJJ5O1oaxY0fpwfh,0.801000,2,0.1280,-7.311,1,True House Music - Xavier Santos & Carlos Gomi...,17,1920-01-01,0.0425,127.997,0.0422,1920
4,4,0.295000,MIXE,0.704,165224,0.707,1,2hJjbsLCytGsnAHfdsLejp,0.000246,10,0.4020,-6.036,0,Xuniverxe,2,1920-10-01,0.0768,122.076,0.2990,1920
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174384,174384,0.009170,"DJ COMBO', 'SANDER-7', 'TONY T",0.792,147615,0.866,0,46LhBf6TvYjZU2SMvGZAbn,0.000060,6,0.1780,-5.089,0,The One,0,2020-12-25,0.0356,125.972,0.1860,2020
174385,174385,0.795000,ALESSIA CARA,0.429,144720,0.211,0,7tue2Wemjd0FZzRtDrQFZd,0.000000,4,0.1960,-11.665,1,A Little More,0,2021-01-22,0.0360,94.710,0.2280,2021
174386,174386,0.806000,ROGER FLY,0.671,218147,0.589,0,48Qj61hOdYmUCFJbpQ29Ob,0.920000,4,0.1130,-12.393,0,Together,0,2020-12-09,0.0282,108.058,0.7140,2020
174387,174387,0.920000,TAYLOR SWIFT,0.462,244000,0.240,1,1gcyHQpBQ1lfXGdhZmWrHP,0.000000,0,0.1130,-12.077,1,champagne problems,69,2021-01-07,0.0377,171.319,0.3200,2021


In [10]:
# adding direct url to data set by adding url prefix and id
url = 'http://open.spotify.com/track/' + df['id']
df['url'] = url

In [11]:
df['combined'] = df['name'] + ' - ' +  df['artists']
df = df.drop_duplicates(subset='combined', keep='first')

In [12]:
# reordering columns, leaving out ID and release date
df = df[['combined', 'url', 'year', 'acousticness', 'danceability', 'duration_ms', 'energy',
       'explicit', 'instrumentalness', 'key', 'liveness', 'loudness',
       'mode', 'popularity', 'speechiness', 'tempo', 'valence']]

In [13]:
df

,combined,url,year,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence
0,Keep A Song In Your Soul - MAMIE SMITH,http://open.spotify.com/track/0cS0A1fUEUd1EW3F...,1920,0.991000,0.598,168333,0.2240,0,0.000522,5,0.3790,-12.628,0,12,0.0936,149.976,0.6340
1,I Put A Spell On You - SCREAMIN' JAY HAWKINS,http://open.spotify.com/track/0hbkKFIJm7Z05H8Z...,1920,0.643000,0.852,150200,0.5170,0,0.026400,5,0.0809,-7.261,0,7,0.0534,86.889,0.9500
2,Golfing Papa - MAMIE SMITH,http://open.spotify.com/track/11m7laMUgmOKqI3o...,1920,0.993000,0.647,163827,0.1860,0,0.000018,0,0.5190,-12.098,1,4,0.1740,97.600,0.6890
3,True House Music - Xavier Santos & Carlos Gomi...,http://open.spotify.com/track/19Lc5SfJJ5O1oaxY...,1920,0.000173,0.730,422087,0.7980,0,0.801000,2,0.1280,-7.311,1,17,0.0425,127.997,0.0422
4,Xuniverxe - MIXE,http://open.spotify.com/track/2hJjbsLCytGsnAHf...,1920,0.295000,0.704,165224,0.7070,1,0.000246,10,0.4020,-6.036,0,2,0.0768,122.076,0.2990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174365,"I giorni - LUDOVICO EINAUDI', 'JOHANNES BORNLÖF",http://open.spotify.com/track/0sZ6HFSulsrOl6VT...,2021,0.976000,0.358,381150,0.1210,0,0.889000,2,0.1350,-25.111,1,0,0.0532,131.808,0.1060
174369,"Nuvole bianche - LUDOVICO EINAUDI', 'JOHANNES ...",http://open.spotify.com/track/2LeqqwzobL5ktfQh...,2021,0.995000,0.297,349200,0.0287,0,0.908000,8,0.0995,-30.008,1,0,0.0564,141.636,0.0678
174371,"Una Mattina - LUDOVICO EINAUDI', 'JOHANNES BOR...",http://open.spotify.com/track/3Glmyv3hbGGTgeR3...,2021,0.995000,0.343,206700,0.0165,0,0.878000,9,0.0774,-30.915,0,0,0.0455,126.970,0.1510
174375,"Night - LUDOVICO EINAUDI', 'JOHANNES BORNLÖF",http://open.spotify.com/track/6QGVWUbmlePAiY5z...,2021,0.988000,0.316,303333,0.0573,0,0.879000,3,0.1200,-24.121,1,0,0.0515,81.070,0.0373


In [15]:
# finding a song I want to use as an input
tay = df[df['combined'] == 'Keep A Song In Your Soul - MAMIE SMITH'].sort_values('popularity', ascending=False)
tay

,combined,url,year,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence
0,Keep A Song In Your Soul - MAMIE SMITH,http://open.spotify.com/track/0cS0A1fUEUd1EW3F...,1920,0.991,0.598,168333,0.224,0,0.000522,5,0.379,-12.628,0,12,0.0936,149.976,0.634


In [16]:
# target set will be both artist and name
y_set = ['combined', 'url']

# droping target from data matrix
df_data = df.drop(y_set, axis=1)

# set target
df_target = df[y_set]

In [17]:
# fit on data, 12 neighbors
nn = NearestNeighbors(algorithm='brute', leaf_size =15, n_neighbors=12, n_jobs=-1)
nn.fit(df_data)

NearestNeighbors(algorithm='brute', leaf_size=15, metric='minkowski',
                 metric_params=None, n_jobs=-1, n_neighbors=12, p=2,
                 radius=1.0)

In [18]:
# sample a song(index) from df_data to use as our query point 
input_index = 0 # Drake - Tootsie slide

# vectorize 
data_vect = [df_data.iloc[input_index].values]
data_vect

[array([ 1.92000e+03,  9.91000e-01,  5.98000e-01,  1.68333e+05,
         2.24000e-01,  0.00000e+00,  5.22000e-04,  5.00000e+00,
         3.79000e-01, -1.26280e+01,  0.00000e+00,  1.20000e+01,
         9.36000e-02,  1.49976e+02,  6.34000e-01])]

In [19]:
# Query Using kneighbors 
neigh_dist, neigh_indices = nn.kneighbors(data_vect)

In [20]:
# top 12 closest data vectors to our reference vector, data_vect
neigh_dist

array([[1.95312500e-03, 3.23694929e+01, 3.55662785e+01, 3.55662785e+01,
        3.56921692e+01, 3.75565655e+01, 3.92994094e+01, 4.14127670e+01,
        4.33780251e+01, 4.67122796e+01, 4.80296392e+01, 4.82701063e+01]])

In [21]:
# these are the corresponding indicies of the most similar vectors 
neigh_indices

array([[     0, 117176,   1573,  55120,  56484,  19840,   1227,  87781,
          6020,     63, 134873,  40034]], dtype=int64)

In [22]:
# breaking down the array to a list
indexs = neigh_indices.flat[0:12].tolist()
indexs

[0, 117176, 1573, 55120, 56484, 19840, 1227, 87781, 6020, 63, 134873, 40034]

In [23]:
# result metrics
df_data.iloc[indexs]

,year,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence
0,1920,0.991,0.598,168333,0.2240,0,0.000522,5,0.379,-12.628,0,12,0.0936,149.976,0.634
127187,1935,0.996,0.509,168323,0.3100,0,0.897000,5,0.406,-11.276,1,0,0.0938,125.994,0.624
1696,1929,0.996,0.743,168347,0.1590,0,0.281000,7,0.150,-14.196,1,0,0.1290,121.057,0.822
59054,1929,0.996,0.743,168347,0.1590,0,0.281000,7,0.150,-14.196,1,0,0.1290,121.057,0.822
60455,1939,0.990,0.642,168333,0.2540,0,0.000038,5,0.199,-13.439,0,0,0.0512,122.259,0.698
21271,1929,0.995,0.690,168347,0.0969,0,0.880000,2,0.107,-19.815,1,0,0.1670,119.530,0.713
1314,1927,0.995,0.791,168320,0.0474,0,0.042000,9,0.101,-17.978,0,0,0.0686,116.247,0.800
94593,1930,0.995,0.530,168314,0.3040,0,0.870000,5,0.645,-17.874,1,0,0.1660,117.103,0.494
6426,1952,0.913,0.657,168360,0.1470,0,0.046100,0,0.116,-21.493,0,16,0.0426,152.971,0.560
63,1920,0.996,0.369,168293,0.2850,0,0.938000,7,0.193,-16.523,1,0,0.0528,129.558,0.635


In [24]:
# result target
df_target.iloc[indexs]

,combined,url
0,Keep A Song In Your Soul - MAMIE SMITH,http://open.spotify.com/track/0cS0A1fUEUd1EW3F...
127187,Şu Kızı Bana Vermiyorlar - MAKBULE ENVER HANIM,http://open.spotify.com/track/74vda4pNQoNuh2DB...
1696,"De a Traicion - FRANCISCO CANARO', 'CHARLO",http://open.spotify.com/track/0824IiWIQJCw9GP8...
59054,De a Traicion - Remasterizado - FRANCISCO CANA...,http://open.spotify.com/track/56HmximYQWTjpghF...
60455,O mia bella Napoli - TITI BOTEZ,http://open.spotify.com/track/4BspizHHJl4dY03B...
21271,Sos Un Hombre - Instrumental (Remasterizado) -...,http://open.spotify.com/track/2vebVasXSfdp7SAO...
1314,¿Cuando Volverás? - Remasterizado - FRANCISCO ...,http://open.spotify.com/track/0NfIlp9ZOS0wHLqt...
94593,Consejo de Amigo - PEDRO MAFFIA,http://open.spotify.com/track/4LWO3ULGwCgcCW8y...
6426,It Could Happen To You - Remastered - NAT KING...,http://open.spotify.com/track/5hU6H5pEbwOa0fLJ...
63,El Baccarat - Remasterizado - FRANCISCO CANARO,http://open.spotify.com/track/1QI5WTvnpNV5vJJQ...


In [27]:
# creates the model into a pickle file
dump(nn, 'model.joblib', compress=True)

['model.joblib']